## <center><strong>Notebook<span style= "color: #51FCC6"> Describtion

## <center><strong>Importing<span style= "color: #48E0DC"> Packeges

In [1]:
import time
import sqlite3
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats

In [2]:
COLORS = ["#51fcc6", "#48e0dc", "#5cd3f7", "#4895e0", "#517afc"]

%matplotlib inline

NUMERICS = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
            'uint16', 'uint32', 'uint64']

set_matplotlib_formats('pdf', 'svg')
# pyo.init_notebook_mode(connected= True)

C:\Users\FreeComp\AppData\Local\Temp\ipykernel_2596\3456799740.py:8: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf', 'svg')


## <center><strong>Setting up the<span style= "color: #5CD3F7"> Web scrapers

In [3]:
fivver_data_jobs_titles = ["data-entry", "data-processing", "data-engineering",
                           "data-science", "data-analytics", "data-visualization"]

guru_data_jobs_titles = ["data-analytics", "machine-learning", "data-visualization"]


upwork_service_types = ["profiles", "jobs", "services"]
guru_service_types = ["freelancers", "jobs"]


def scrape_page(url: str) -> str:

    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.page_source

    driver.quit()
    return  html

In [31]:
# The `type` param indicates what is the nature of the job is it hiring freelancers 
# or paying services or othr thing based on the platform.

def upwork_scraper(page, job_title, type):

    global upwork_url

    if  type == 'services':
        
        upwork_url =  f"https://www.upwork.com/services/search?nbs=1&q={job_title}"

        driver = webdriver.Chrome()
        driver.get(upwork_url)

        if page > 1:
            for i in ragne(page):
                
                wait = WebDriverWait(driver, 10)
                element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.up-btn.up-btn-default.mx-0.load-more-btn.nuxt-link-active')))
                element.click()
                
                time.sleep(1)

        html = driver.page_source
        driver.quit()
        
        return  html

    elif type == "profiles" or type == "jobs":
        
        upwork_url = f"https://www.upwork.com/search/{type}/?page={page}&q={job_title}&user_pref=1"
        
        return scrape_page(upwork_url)

        
def guru_scraper(page, job_title, type):

    global guru_url
    guru_url = f"https://www.guru.com/d/{type}/c/programming-development/sc/math-algorithms-analytics/ssc/{job_title}/pg/{page}/"
    
    return scrape_page(guru_url)


def fivver_scraper(page, job_title):

    global fivver_url
    fivver_url = f"https://www.fiverr.com/categories/data/{job_title}?source=pagination&page={page}"

    return scrape_page(fivver_url)


def peopleperhour_scraper(page, job_title):

    global peopleperhour_url
    peopleperhour_url = f"https://www.peopleperhour.com/services/{job_title.replace('-', '+')}?page={page}&ref=search"

    return  scrape_page(peopleperhour_url)

# We will not analyze requested jobs data in freelancer.com becuase it's search feature 
# isn't accurate with the it.

def freelancer_scraper(page, job_title):

    global freelancer_url
    freelancer_url = f"https://www.freelancer.com/search/users?q={job_title.replace('-', '%20')}&page={page}"
    
    return scrape_page(freelancer_url)

Now we will try to calculate how much did each scraper take to collect the data

In [32]:
get_variable_name = lambda var: [name for name in globals() if globals()[name] is var][0]

def get_runtime(func, *args, **kwargs):

    t1 = time.perf_counter()
    output = func(*args, **kwargs)
    t2 = time.perf_counter()

    run_time = t2 - t1

    print("{} took about {:.2f}s to run.\n".format(
        get_variable_name(func).replace("_", " ").title(), run_time))

    return  output

Now we will just try to scrape **only one page** from each scraper *(temporary action)* to just test the data extractors with<br>
more efficient way.

In [33]:
# If the job type is services in upwork the pages paremeter will refer to how many pages 
# will render at once while the other scrapers will render the spesfic spesfied page only.


upwork_html: str = get_runtime(upwork_scraper,1 , fivver_data_jobs_titles[4], "services")
guru_html: str = get_runtime(guru_scraper, 1, guru_data_jobs_titles[2], "freelancers")

fivver_html: str = get_runtime(fivver_scraper, 1, fivver_data_jobs_titles[4])
peopleperhour_html: str = get_runtime(peopleperhour_scraper, 1, fivver_data_jobs_titles[4])
freelancer_html: str = get_runtime(freelancer_scraper, 1, fivver_data_jobs_titles[4])

MaxRetryError: HTTPConnectionPool(host='localhost', port=57973): Max retries exceeded with url: /session/0f64e85c3f3e86bb85f15eda882b2ef0/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000162C8D3BEB0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [23]:
upwork_url
# https://www.upwork.com/services/search?nbs=1&q=data%20analysis

'https://www.upwork.com/search/services/?page=1&q=data-analytics&user_pref=1'

## <center><strong>Extracting the <span style = "color: #4895e0"> Scrapers </span> data

In [12]:
test_pages = [upwork_html,
             fivver_html,
             guru_html,
             peopleperhour_html,
             freelancer_html]

soups = {"soup_name": [],
         "soup_url": [],
         "soup": []}

for page in test_pages:

    soup = BeautifulSoup(page)

    soups["soup"].append(soup)
    soups["soup_name"].append(get_variable_name(page).replace("html",
                                                              "soup"))
    
    soups["soup_url"].append(eval(get_variable_name(page)[:-5] + "_url"))

In [16]:
print("Printing the title of each page to check if it's loaded correctly.\n")

for soup in soups["soup"]:
    print("-\t", soup.title.text.replace("\n", "").replace("\t", ""))

Printing the title of each page to check if it's loaded correctly.

-	 404: Page Not Found - Upwork
-	 Data Analytics Services Online | Fiverr
-	 Find and Hire Freelancers for Data Visualization - Guru 
-	 Get Data Analytics Offers & Services | PeoplePerHour
-	 Users Jobs, Employment | Freelancer


Looks like everything is working as expected now let's start reel work by extracting the HTML tags data from the scraped<br>
pages.<br> <br>

But we will need to collect the data from each one of those pages with unique way further more we need to optimize the<br>
data extractor to collect the data from other pages from the same site like "jobs" site and "profiles" sites on upwork.<br>

In [113]:
soups = pd.DataFrame(soups)

In [114]:
soups["soup_url"]

0    https://www.upwork.com/search/services/?page=1...
1    https://www.fiverr.com/categories/data/data-an...
2    https://www.guru.com/d/freelancers/c/programmi...
3    https://www.peopleperhour.com/services/data+an...
4    https://www.freelancer.com/search/users?q=data...
Name: soup_url, dtype: object

In [87]:
soups["soup_name"]

0           upwork_soup
1           fivver_soup
2             guru_soup
3    peopleperhour_soup
4       freelancer_soup
Name: soup_name, dtype: object

In [88]:
def get_upwork_links(soup_name, job_type: str) -> list:  

    card_tag = "a"
    link_template = "https://www.upwork.com###"
    link_attr = "href"

    if job_type == "profiles":

        card_tag = "div"
        card_tag_class = "up-card-section up-card-hover"
        link_template = "https://www.upwork.com/search/profiles/?q=profiles&profile=###"
        link_attr = "data-test-key"
    
    elif job_type == "jobs":
        card_tag_class = "up-n-link"


    elif job_type == "services":
        card_tag_class = "up-n-link project-tile up-card m-0 p-0 up-card-section up-card-hover project-tile_flexible"

    else:
        raise Exception("Please input correct work type from ['profiles', 'jobs', 'services'].")


    soup = soups[soups["soup_name"] == soup_name]["soup"][0]

    cards = soup.find_all(card_tag, class_= card_tag_class)
    cards_ids = []

    for card in cards:
        
        card_id = link_template.replace("###", card[link_attr])
        
        if len(card_id) > len(link_template):
            cards_ids.append(card_id)

    return  cards_ids

In [92]:
get_upwork_links("upwork_soup", "services")

[]

In [51]:
len(link_template)

NameError: name 'link_template' is not defined

In [18]:
soups["soup_url"][4]

'https://www.freelancer.com/search/users?q=data%20analytics&page=1'

In [24]:
soups[soups["soup_name"] == "upwork_soup"]["soup"][0]

<html lang="en" theme="air-2-76"><head><script async="" src="//static.ads-twitter.com/uwt.js"></script><script async="" src="//bat.bing.com/bat.js"></script><script async="" id="pdst-capture" src="https://cdn.pdst.fm/ping.min.js"></script><script async="" src="https://d.impactradius-event.com/A2798569-68a2-43d5-98ff-96c66d56992b1.js"></script><script async="" src="https://www.redditstatic.com/ads/pixel.js" type="text/javascript"></script><script async="" src="https://connect.facebook.net/en_US/fbevents.js" type="text/javascript"></script><script async="" src="https://snap.licdn.com/li.lms-analytics/insight.min.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=AW-972598239&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://connect.facebook.net/signals/config/816554411748126?v=2.9.107&amp;r=stable"></script><script async="" src="https://connect.facebook.net/signals/config/463930381624031?v=2.9.107&am

In [97]:
soups[soups["soup_name"] == "upwork_soup"]["soup"][0]

<html lang="en-US" theme="air-3-0"><head>
<meta content="width=device-width, initial-scale=1, minimum-scale=1" name="viewport"/>
<meta charset="utf-8"/>
<title>404: Page Not Found - Upwork</title>
<link href="https://upwork.com/favicon.ico" rel="icon"/>
<link href="https://www.upwork.com/static/fonts-global/4.0.0/fonts.css" rel="stylesheet"/>
<script>window.__SUIT_CONFIG__={pageLocation:"lb:error:404"};</script>
<script async="" src="https://www.upwork.com/static/suit2-tracker/0.2.56/index.lb.iife.min.js"></script>
<style>[theme=air-3-0]{--ws-mobile-x:var(--ws-4x);--ws-heading:.5em;--height-base:40px;--height-sm:32px;--width-sm:218px;--max-width-global-container:1600px;--font-family-base:"Neue Montreal";--font-family-rebrand:Rza,Rza-fallback;--font-size-base:16px;--line-height-base:22px;--font-size-base-sm:14px;--line-height-base-sm:20px;--font-size-base-xs:11px;--line-height-base-xs:16px;--font-size-h1:56px;--line-height-h1:1;--font-size-h2:40px;--line-height-h2:42px;--font-size-h3:32